# PHYS 2600 - Final Project 2: `gravity`

## Motion of Small Objects in the Solar System

This is one possible _final project_; __you should choose one (and only one) project to complete and hand in.__  (If you try to work on two, I will grade the more complete one.  But these are big projects so I very strongly do NOT recommend attempting both!)

The deadline for this project is __7:00 PM (Boulder time) on Thursday, December 11.__ Since final grades must be submitted to the university shortly after the end of the exam period, to allow for timely grading _no late submissions will be accepted._

You are permitted to discuss the final projects with other students in the class,  including both the physics and computing aspects, but __your submitted project must be entirely your own__; no direct copying of Python code is permitted, and you must write up the presentation in your own words.

Your completed project will consist of __two (2) files__:

* `gravity.py`: a Python module (see lecture 25) which implements the functions listed in the Application Progamming Interface (API) below.
* `gravity_presentation.ipynb`: a Jupyter notebook which uses the code from your Python module to answer the physics questions.  Use of Markdown text, MathJax equations, and plots are all encouraged to make your presentation clear and compelling!


The rubric for grading will be split 50/50 between the code (in your `.py` module and Jupyter notebook) and the response to the physics questions (provided in `gravity_presentation.ipynb`):
* Basic functionality of code (passes provided tests, follows API specification): __30%__
* __Six (6)__ additional API tests _you write_ (tests are provided and are correct and useful): __10%__
* Documentation of code (docstrings and comments make the code clear): __10%__
* Correct and detailed answers to physics questions: __40%__
* Quality of presentation (clear and readable, appropriate use of Markdown, equations, and plots): __10%__

A _bonus of up to 10%_ is available for the extra-credit "challenge" physics question at the end.  (These challenge questions are meant to be really hard!  But partial credit will be awarded if you make some progress on them.)

# Overview

The motion of large objects in the solar system is very well understood: Kepler's laws of planetary motion are one of the first applications of the law of gravitation that most physics students see.  However, understanding dynamics under gravity is a very difficult problem in general: even though the motion of _two_ objects can be solved on pen and paper, the "three-body problem" (motion of three massive objects under gravity) already has no analytic solutions.  So tracking a single object as it moves through the gravitational field of the solar system is not doable with pen and paper - making it a great candidate for numerical study.

For this project, you'll begin by modeling the Sun and the planets of the solar system in their known orbits. Then, you will write a code which can solve for the motion of a small object through the complex gravity field of the Sun and the planets.  (To make things a little less complicated, we will only consider the gravity of Jupiter, the largest planet, in addition to the Sun.)  As one application, you will study the motion of 'Oumuamua - the first interstellar object recorded passing through our solar system.  Then you will consider the paths of comets, looking for which ones will pass close enough to Earth to threaten a collision - and which ones are "swept up" by Jupiter before they can reach the inner solar system.


# Physics background and important equations

Everything starts with the law of gravitation: the force on a small mass $m$ due to large mass $M$ at the origin is

$$
\vec{F}_g = -\frac{GmM}{r^3} \vec{r}
$$

where $\vec{r} = (x,y,z)$ is the position vector of $m$ with respect to $M$, and the minus sign appears since gravity is attractive (so the force is always towards the origin.)  

For this project, we'll have multiple sources of gravity to worry about, so this is better written in the more general form (cancelling the mass $m$ out)

$$
\ddot{\vec{r}} = -\frac{GM}{|\vec{r} - \vec{R}|^3} (\vec{r} - \vec{R})
$$

where $\mathbf{r}$ is the position vector of the test mass $m$, and $\vec{R}$ is the position vector of the gravity source $M$.  The double dots denote two time derivatives, i.e. $\ddot{\vec{r}} = d^2 \vec{r} / dt^2$.

### Model of the solar system

Before we can study the motion of small objects, we need a model for the solar system itself.  Assuming all the planets are much lighter than the Sun, the planets follow elliptical orbits around the Sun of the form

$$
r(\phi) = \frac{a (1 - \epsilon^2)}{1+\epsilon \cos (\phi - \omega)}
$$
where $a$ is the semimajor axis of the orbit, $\epsilon$ is the eccentricity (how un-circular the orbit is), and $\phi$ is the orbital phase.  The angle $\omega$ determines the location of __perihelion__ (closest approach to the Sun), and is different for every planet.  

To determine $\phi$ as a function of time, we just use Kepler's second law, which can be written as

$$
\frac{d\phi}{dt} = \frac{L_z}{\mu r^2} \approx \frac{\sqrt{GM_{\odot} a (1 - \epsilon^2)}}{r^2}
$$

where $M_{\odot}$ is the mass of the Sun,

$$
M_{\odot} = 1.9885 \times 10^{30}\ {\rm kg}.
$$

For non-circular orbits the equation for $d\phi / dt$ leads to tricky integrals that can't be done in closed form, but we don't care since we're doing numerics: we'll just use this equation directly to update $\phi$ at each timestep, given the current value of $r$.  Assuming all eight planets are in the same plane (which is a good approximation: Mercury is 7 degrees out of the ecliptic plane, but it's also a small planet!), their coordinates as a function of time are then just

$$
\vec{R}(\phi) = (r(\phi) \cos \phi, r(\phi) \sin \phi, 0)
$$


| planet | mass ($\times 10^{24}$ kg) |  a (AU) | $\epsilon$ | $\omega$ (${}^{\circ}$) | $\phi_0$ (${}^{\circ}$) [11/16/22] |
|--------|------|-----|------------|-------------------|-----|
| Mercury | 0.330104 | 0.38709843 | 0.20563661 | 29.11810076 | -150.643358 |
| Venus | 4.86732 | 0.72332102 | 0.00676399 | 55.09494217 | 95.736269 |
| __Earth__ | 5.97219 | 1.00000018 | 0.01673163 | 108.04266274 | -45.2075738 |
| Mars | 0.641693 | 1.52371243 | 0.09336511 | -73.63065768 | 11.6829464 |
| __Jupiter__ | 1898.13 | 5.20248019 | 0.04853590 | -86.0178741 | 11.5345120 |
| Saturn | 568.319 | 9.54149883 | 0.05550825 | -20.77862639 | -25.052655 |
| Uranus | 86.8103 | 19.18797948 | 0.04685740 | 98.47154226 | 47.2232752 |
| Neptune | 102.41 | 30.06952752 | 0.00895439 | -85.10477129 | -0.14322895 |

Orbital parameters were taken from NASA's Jet Propulsion Laboratory (there used to be a link here, but they reorganized their website and the link is broken.)  Masses are from NASA's [Solar System Exploration site](https://solarsystem.nasa.gov/planet-compare/).  _(Note: I did some coordinate conversions myself to put this into our coordinate system.  No warranty is given that these coordinates will perfectly match other sources you may find.)_

Another note: I give the angles above in degrees, but it's annoying to include degree-radian conversions everywhere, so you should __work with angles in radians__ internally in your code.

# Computational Strategy and Algorithms

This is actually a deceptively hard numerical problem - if we just try to use a simple Euler's method solution for the planets and the small test mass, our code will be either very slow or very inaccurate!  The problem is that for orbits with a high eccentricity, a very small fraction of the orbital period will be spent in the part of the orbit near the Sun, where the acceleration is largest.  We need a small time-step to deal with this close region, but that means we'll need a _lot_ of time steps to get through the entire orbit!

As a result, we will try to be as efficient as possible in our simulation, which means choosing good algorithms and using NumPy arrays and vectorized operations whenever we can.


We have two separate models to keep track of.  For the model of the solar system, we calculate a numerical value for $d\phi / dt$ for each planet.  Given a timestep $dt$, we can use Euler's method to update the angular positions $\phi$, and then compute $r(\phi)$ from above.  Here Euler's method is good enough, because imposing Kepler's laws makes sure the orbits are stable against discretization errors.




For the motion of the small mass itself, since we have a single second-order equation, we use the familiar trick of splitting it into two equations:

$$
\vec{v} = \frac{d\vec{r}}{dt} \\
\frac{d\vec{v}}{dt} = \ddot{\vec{r}} = -\sum_{M} \frac{GM}{|\vec{r} - \vec{R}|^3} (\vec{r} - \vec{R})
$$

(Actually, it's six equations since $\mathbf{r}$ and $\mathbf{v}$ are both three-vectors, but it's the same trick.)  Then we update by discrete timesteps, but we'll use an improvement on Euler's method specialized to problems like this called the __leapfrog method:__ our updates will take the form

$$
\vec{r}_{1/2} = \vec{r}_{i} + \frac{1}{2} \vec{v}_{i} \Delta t \\
\vec{v}_{i+1} = \vec{v}_i + \ddot{\vec{r}}_{1/2} \Delta t \\
\vec{r}_{i+1} = \vec{r}_{1/2} + \frac{1}{2} \vec{v}_{i+1} \Delta t
$$

You can read a bit more about leapfrog methods [in these notes](https://bpb-us-e1.wpmucdn.com/sites.ucsc.edu/dist/7/1905/files/2025/03/leapfrog.pdf); the alternating of "half-step" updates for velocity and position leads to much better accuracy, and in particular it does much better with conservation of energy when evolving a physical system.  (In fact, the code I will give you is a more complicated version of leapfrog called the "Omelyan SST" integrator - this problem can be computationally intensive, so I want to make sure you can have the smallest step-size errors possible!)

That's all we need for the basic functionality of this project!  The computational setup here is relatively simple for a single object - keeping track of many objects at once and doing data analysis on the outputs are where the difficulty lies...

# Application Programming Interface (API) specification

__You must implement all of the below functions, according to the specifications given.__  

I actually recommend _not_ using Unyt for the computationally intensive parts of this program: it will slow down the code significantly!  You may find it helpful to use Unyt to calculate numerical values in a certain unit system when you're initializing the code, however.  I used "kg-AU-s" units, but you can choose what you like - just keep your units consistent!

A commonly-used array structure appearing in our API will be the __planet array__:

```
planets = np.array([
    [a1, eps1, omega1, phi1],
    [a2, eps2, omega2, phi2],
    ...
])
```
This is an $(N_p \times 4)$ array containing the orbital properties of the planets and their current angular positions $\phi$ __in radians__.  (Usually $N_p$ will be 8, but you may want to omit or add things for certain tests.)


### Planets:

* `load_planets(planet_file)`: Given the file `planets.csv`, returns a planet array as described above.
* `get_planet_r(planets)`: Given a planet array, returns a length $N_p$ one-dimensional array of the planets' current distances from the Sun $r(\phi)$.
* `get_planet_coords(planets)`: Given a planet array, returns an $(N_p \times 3)$ array of their current three-dimensional Cartesian coordinates $(x,y,z)$.
* `get_planet_orbits(planets, phi_linspace)`: Given a planet array and a NumPy array of phi values, returns a length-$N_p$ list; each entry in the list should be a tuple of two arrays `(x_array, y_array)` of the same length as `phi_linspace`, containing the $x$ and $y$ coordinates of each planet at each phi value.  (This is a sort of complicated structure, but it's divided up like this to allow you to easily loop over each planet and plot its orbit using `matplotlib`.)
* `update_all_planets(planets, dt)`: Given a planet array and a timestep `dt`, uses as single Euler step to update the planets' positions `phi`.  __Works by mutation__ - this should change the `planets` array __in-place__ and return the `None` object.

### Small mass:

* `accel_g_sun(vec_r)`: Given a position three-vector `vec_r` = $(x,y,z)$, computes and returns the gravitational acceleration due to the Sun as a (NumPy) three-vector.
* `accel_g_jupiter(vec_r, planets)`:  Same as `accel_g_sun(vec_r)`, but returns the acceleration (NumPy) three-vector due to the planet Jupiter instead.

### Other functions:

* `get_planet_distances(vec_r, planets)`: Given a position three-vector `vec_r` and a list of planets, returns a length-$N_p$ NumPy array containing the distances from `vec_r` to each planet (in the same order as in `planets`).
* `run_API_tests()`: A custom function that should use assertions to test the other functions implemented in the API.  If all tests are passed, the function should simply return the `None` object.  You should implement __at least six (6) tests__ inside this function on your own.

## Main loop

The below code implements the "main loop", finding the trajectory of a small object given its initial position `vec_r0`, initial velocity `vec_v0`, an initial planet array, and a NumPy linspace `t_steps` containing the full time range to simulate over, separated by the desired timestep `dt`.  It also has a switch variable called `sun_only`: setting this to `False` will calculate and add the acceleration vector due to Jupiter.

Once you have implemented the API above, add this code to your `gravity.py` module, then call it to run the simulation in your notebook!

In [ ]:
def accel_total(vec_r, planets, sun_only=False):
    """
    Helper function for find_trajectory below.
    """
    if sun_only:
        return accel_g_sun(vec_r)
    else:
        return accel_g_sun(vec_r) + accel_g_jupiter(vec_r, planets)


def find_trajectory(vec_r0, vec_v0, planets, t_steps, sun_only=True):
    """
    Main loop for solar system gravitation project.

    Arguments:
    =====
    * vec_r0: Initial 3-vector position of the small mass (Cartesian coordinates.)
    * vec_v0: Initial 3-vector velocity of the small mass (Cartesian coordinates.)
    * planets: an (Np x 4) planet array, at their initial positions - see API for description.
        Each row in a planet array contains the values:
            (a, eps, omega, phi)
        where phi is the planet's angular position,
        and a, eps, omega are orbital parameters.
    * t_steps: NumPy array (linspace or arange) specifying the range of times to simulate
        the trajectory over, regularly spaced by timestep dt.
    * sun_only: binary flag.  If set to True, only the Sun's gravity is included in the simulation.
        If False, then Jupiter's acceleration is included as well.

    Returns:
    =====
    A tuple of the form (r, planet_distance, v).

    "r" contains the coordinates (x,y,z) of the test mass at each
    corresponding time in t_steps, as a (3 x Nt) array.
    "planet_distance" contains the distances from the small mass
    to each planet in planets, in order, as a function of time - this is a
    (Np x Nt) array.
    "v" contains the velocity vector of the test mass at each time
    in t_steps, as a (3 x Nt) array.

    """

    dt = t_steps[1] - t_steps[0]
    Nt = len(t_steps)

    r = np.zeros((3, Nt))
    r[:, 0] = vec_r0

    v = np.zeros((3, Nt))
    v[:, 0] = vec_v0

    planet_distance = np.zeros((8, Nt))
    planet_distance[:, 0] = get_planet_distances(vec_r0, planets)

    # Copy the planets array so we don't change the original!
    local_planets = planets.copy()

    for i in range(Nt - 1):
        ## Omelyan SST update

        ## V dt/6
        update_all_planets(local_planets, dt / 6)
        vec_v = v[:, i] + (dt / 6) * accel_total(
            r[:, i], local_planets, sun_only=sun_only
        )

        ## T dt/2
        vec_r = r[:, i] + vec_v * dt / 2

        ## V 2dt/3
        update_all_planets(local_planets, 2 * dt / 3)
        vec_v = vec_v + (2 * dt / 3) * accel_total(
            vec_r, local_planets, sun_only=sun_only
        )

        ## T dt/2; final position update
        r[:, i + 1] = vec_r + vec_v * dt / 2

        ## V dt/6; final velocity update
        update_all_planets(local_planets, dt / 6)
        v[:, i + 1] = vec_v + (dt / 6) * accel_total(
            r[:, i + 1], local_planets, sun_only=sun_only
        )

        planet_distance[:, i + 1] = get_planet_distances(r[:, i + 1], local_planets)

    return (r, planet_distance, v)

---
---
---

The below appendix is included for your information, but you don't need to understand it in detail for the project.

### Appendix A: Velocity and planetary orbits

Here, I work out some additional details for planetary orbits, based on the two orbital equations

$$
r(\phi) = \frac{a (1 - \epsilon^2)}{1+\epsilon \cos (\phi - \omega)}
$$

and

$$
\frac{d\phi}{dt} = \frac{L_z}{\mu r^2} \approx \frac{\sqrt{GM_{\odot} a (1 - \epsilon^2)}}{r^2}.
$$

One useful piece of information for testing will be to compute the __velocity vector__ $\mathbf{V}$ of a planet.  We can find this by taking the time derivative of $\mathbf{R}$:

$$
\mathbf{V} = \frac{d\mathbf{R}}{dt} = (\dot{r} \cos \phi - r \dot{\phi} \sin \phi, \dot{r} \sin \phi + r \dot{\phi} \cos \phi, 0)
$$

From above, the derivative of $r(\phi)$ is
$$
\frac{dr}{dt} = \frac{a(1-\epsilon^2) (-\epsilon \sin(\phi - \omega))}{(1+\epsilon \cos (\phi - \omega))^2} \frac{d\phi}{dt} = -r^2 \frac{\epsilon \sin(\phi - \omega)}{a(1-\epsilon^2)} \frac{d\phi}{dt}
$$


If we really want $\mathbf{V}(\phi)$, this is enough to find it numerically - it's pretty messy otherwise.  However, it's much simpler if we ask for the _maximum speed_ instead.  For a planetary orbit, maximum speed occurs at perihelion, $\phi = \omega$.  Plugging this in drastically simplifies the equation for velocity, because if $\phi = \omega$ we have $dr/dt = 0$ from above.  Thus, we're left with:

$$
\mathbf{V}_{\rm max} = (-r(\omega) \dot{\phi}(\omega) \sin \omega, r(\omega) \dot{\phi}(\omega) \cos \omega, 0)
$$

and the __maximum speed__ is
$$
|\mathbf{V}_{\rm max}| = r(\omega) \dot{\phi}(\omega) = \frac{\sqrt{GM_{\odot} a (1-\epsilon^2)}}{r(\omega)} = \sqrt{\frac{GM_{\odot} (1+\epsilon)}{a(1-\epsilon)}}.
$$

This can be helpful in determining time-steps for your simulation: if your time resolution is $dt$, the the largest distance error you will make in computing the orbit is $dr = |\mathbf{V}_{\rm max}| dt$.  (When the speed is smaller, the distance traveled in $dt$ is smaller too.)
